# 데이터 세트
https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=528

### import Libraries
tensorflow😂❤️

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

### 데이터 셋 정보

train : 10,025 이미지<br>
validation : 1,539 이미지<br>
test : 1,504 이미지<br>

In [ ]:
label_to_age={
    'a':"0-6 years old",
    'b':"7-12 years old",
    'c':"13-19 years old",
    'd':"20-30 years old",
    'e':"31-45 years old",
    'f':"46-55 years old",
    'g':"56-66 years old",
    'h':"67-80 years old"
}

train_meta_data_path = "./asset/dataset/custom_train_dataset.csv"
train_meta_data = pd.read_csv(train_meta_data_path)
train_image_directory = "./asset/dataset/train_images"

val_meta_data_path = "./asset/dataset/custom_val_dataset.csv"
val_meta_data = pd.read_csv(val_meta_data_path)
val_image_directory = "./asset/dataset/val_images"

test_meta_data_path = "./asset/dataset/custom_test_dataset.csv"
test_meta_data = pd.read_csv(test_meta_data_path)
test_image_directory = "./asset/dataset/test_images"

In [ ]:
train_meta_data['image_path']='./asset/dataset/train_images/'+train_meta_data['image_path']
val_meta_data['image_path']='./asset/dataset/val_images/'+val_meta_data['image_path']
test_meta_data['image_path']='./asset/dataset/test_images/'+test_meta_data['image_path']